In [10]:
#THaW Project
#This program is meant to establish the accuracy of predicting device name from the data collected by MSU
#6/12/2019
#Code Written By: Manzi Bryan with a lot of help from https://www.kaggle.com/nageshnaik/iris-dataset-classfication-using-naive-bayes
# This was used for the final submission
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import *
import sklearn
from sklearn import model_selection
from sklearn.model_selection import *
import pylab as pl
import pandas as pd
import os
import math
import warnings
import pickle
import operator
import time

target = 'Model' 


table = pd.DataFrame(index = [0], columns = [ target, '# train', '# test', 'Accuracy',
                                                     'Avg Conf', 'Std Conf' ])

# This method is meant to build a Random Forest Classifier given a dataframe with devices as targets
# The dontTrainOn parameter is meant to tell the classifier to leave one tenth of one device out of training

def classify(df, function, dontTrainOn, k):
    save = df[df[target] == dontTrainOn]
    numReadings = len(save)
    
    df = df[df[target] != dontTrainOn] # Drop everything we do not want to train on
    ratio = 10
    ninety = [df, save[save.index % ratio != k]]
    df = pd.concat(ninety) # Concatenate one tenth of readings on the dataframe
    devices = df[target].unique()
    i = 0
    df[target].replace(devices, range(0, len(devices)), inplace=True)
        
    Y = df[target].tolist()
    
#     print("There are " + str(len(devices)) + " devices")
    copydf = df.copy()
    
    #Remove labelling columns from the index
    copydf = copydf.drop(columns= ['Device' ,'Model', 'App', 'Distance', 'Bursts'])
    X = copydf.values
    
    #Initialize classifiers
    clf = function
    
    
    #One tenth of the data as test
    validation_size = 0.1
    
    seed = 7
    
    X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, 
                                                    test_size=validation_size, random_state=seed)
    
    scoring = {'acc': 'accuracy',
           'prec_macro': 'precision_macro',
           'rec_micro': 'recall_macro'}
    
    #Fitting the training set
    clf.fit(X_train, Y_train) 
    
    
#     numFeatures = len(X_validation[0])#includes subRegion as a feature
    
   
    #Model Performance
#     #setting performance parameters
#     kfold = model_selection.StratifiedKFold(n_splits=11, random_state=seed) #same number of samples from each 

#     #calling the cross validation function
    
#     cv_results = cross_validate(clf, X_train, Y_train, cv=kfold, scoring=scoring, return_train_score=True)
    
    
    
    
    save = save[save.index % ratio == k]
    makePrediction(clf, save, devices, numReadings, dontTrainOn)

def looper(df):
    global table
    
    classifierDict = {}
    classifier = RandomForestClassifier(bootstrap= True, max_depth= 15, max_features= 3, min_samples_leaf= 1, min_samples_split= 2, n_estimators= 15, random_state=42)
    classifier3 = DecisionTreeClassifier(max_depth= 21, max_features= 3, min_samples_leaf= 1, min_samples_split= 3, random_state=42)
    classifier2 = KNeighborsClassifier(algorithm= 'auto', leaf_size= 1, n_neighbors= 11, weights= 'distance')
    classifier4 = GaussianNB()
    
    classifierDict['RandomForest'] = classifier
    classifierDict['DecisionTree'] = classifier3
    classifierDict['KNN'] = classifier2
    classifierDict['GNB'] = classifier4
    
    devices = df[target].unique()
    
    df.drop(df.columns[df.columns.str.contains('Unnam',case = False)],axis = 1, inplace = True)#Remove unnecessary columns from the index
    
    for classifierName in classifierDict.keys():
        for device in devices:
            k = 0
            for k in range(10):
                print(classifierName + " " + device + str(k))
                classify(df, classifierDict[classifierName], device, k)
        table.to_excel('table' + target + classifierName +  '.xlsx')
        
        table = pd.DataFrame(index = [0], columns = [ target, '# train', '# test', 'Accuracy',
                                                     'Avg Conf', 'Std Conf' ])   
    return devices
    

def getNumerical(devices, y):
    i = 0
    
    for device in devices:
        if device == y[0]:
            return [i] * len(y)
        i += 1

def makePrediction(clf, unseen, devices, numReadings, deviceName):
    unseen = unseen.reindex(sorted(unseen.columns), axis=1)
    
    unseen.drop(unseen.columns[unseen.columns.str.contains('Unname',case = False)],axis = 1, inplace = True)
    
    y = unseen[target].tolist() #unseen should only contain one device, hence y is a list of the same device repeated
    
    
    unseen = unseen.drop(columns= ['Device' ,'Model', 'App', 'Distance', 'Bursts'])
    
    predictions = clf.predict(unseen)
    
    predictions_proba = clf.predict_proba(unseen)
    i = 0
    other = 0
    # Building Prediction table one prediction at a time
    for i in range(0, len(predictions_proba)):
        confidence = max(predictions_proba[i])
        addToTable(devices, y, predictions, numReadings, deviceName, confidence)
    
    
# This method is meant to help build the results table
def addToTable(devices, y, predictions, numReadings, deviceName, confidence):
    global table
    smallTable = pd.DataFrame(index = [0], columns = [target, '# train', '# test', 'Accuracy',
                                                      'Avg Conf', 'Std Conf'])
    
    correctDevice = getNumerical(devices, y)
    
    smallTable[target][0] = deviceName
    
    smallTable['# train'][0] = numReadings - len(predictions)
    
    smallTable['# test'][0] = len(predictions)
    
    smallTable['Accuracy'][0] = accuracy_score(correctDevice, predictions)
    
    smallTable['Avg Conf'][0] = confidence
    
    table = table.append(smallTable)
    
    
if __name__ == "__main__":
    
    path= 'efficientTrain.xlsx'
    
    print("About to read...")
    df = pd.read_excel(path) #Comment this out to run much faster
    print("...read")
    
    df = df.reindex(sorted(df.columns), axis=1)
    df.reindex(range(len(df.Bursts)))
    
    
    looper(df) # comment this out to run faster
    
    
    
    
    
    print("Done")
    

    

About to read...
...read
RandomForest Gian Iphone0
RandomForest Gian Iphone1
RandomForest Gian Iphone2
RandomForest Gian Iphone3
RandomForest Gian Iphone4
RandomForest Gian Iphone5
RandomForest Gian Iphone6
RandomForest Gian Iphone7
RandomForest Gian Iphone8
RandomForest Gian Iphone9
RandomForest Portable ECG moniter 10
RandomForest Portable ECG moniter 11
RandomForest Portable ECG moniter 12
RandomForest Portable ECG moniter 13
RandomForest Portable ECG moniter 14
RandomForest Portable ECG moniter 15
RandomForest Portable ECG moniter 16
RandomForest Portable ECG moniter 17
RandomForest Portable ECG moniter 18
RandomForest Portable ECG moniter 19
RandomForest Bodimetrics Performance monitor0
RandomForest Bodimetrics Performance monitor1
RandomForest Bodimetrics Performance monitor2
RandomForest Bodimetrics Performance monitor3
RandomForest Bodimetrics Performance monitor4
RandomForest Bodimetrics Performance monitor5
RandomForest Bodimetrics Performance monitor6
RandomForest Bodimetric

DecisionTree Eko Sthethoscope0
DecisionTree Eko Sthethoscope1
DecisionTree Eko Sthethoscope2
DecisionTree Eko Sthethoscope3
DecisionTree Eko Sthethoscope4
DecisionTree Eko Sthethoscope5
DecisionTree Eko Sthethoscope6
DecisionTree Eko Sthethoscope7
DecisionTree Eko Sthethoscope8
DecisionTree Eko Sthethoscope9
KNN Gian Iphone0
KNN Gian Iphone1
KNN Gian Iphone2
KNN Gian Iphone3
KNN Gian Iphone4
KNN Gian Iphone5
KNN Gian Iphone6
KNN Gian Iphone7
KNN Gian Iphone8
KNN Gian Iphone9
KNN Portable ECG moniter 10
KNN Portable ECG moniter 11
KNN Portable ECG moniter 12
KNN Portable ECG moniter 13
KNN Portable ECG moniter 14
KNN Portable ECG moniter 15
KNN Portable ECG moniter 16
KNN Portable ECG moniter 17
KNN Portable ECG moniter 18
KNN Portable ECG moniter 19
KNN Bodimetrics Performance monitor0
KNN Bodimetrics Performance monitor1
KNN Bodimetrics Performance monitor2
KNN Bodimetrics Performance monitor3
KNN Bodimetrics Performance monitor4
KNN Bodimetrics Performance monitor5
KNN Bodimetrics Pe